In [1]:
##Importing libraries 

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from   tensorflow.examples.tutorials.mnist import input_data


In [2]:
## Reading the data
df=input_data.read_data_sets("/tmp/data/",one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
###Neural Network Parameters
### 28*28=784

num_inputs=784
num_classes=10
dropout=0.6

In [4]:
###Tensorflow data inputs

X=tf.placeholder(tf.float32,shape=[None,num_inputs])
Y=tf.placeholder(tf.float32,shape=[None,num_classes])
keep_prob=tf.placeholder(tf.float32)


In [5]:
### Neural Network training parameters

batch_size=200
num_steps=400
learning_rate=1e-3
display_step=20

In [6]:
###Creating Network Architechture

def convolution_img(x,W,b,strides=1):
    x=tf.nn.conv2d(x,W,strides=[1,strides,strides,1],padding="SAME")
    x=tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def max_pooling_layer(x,k=3):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding="SAME")




In [7]:
###Model Architechture Weights


W={"w1":tf.Variable(tf.random_normal([5,5,1,32])),
   "w2": tf.Variable(tf.random_normal([5,5,32,64])),
   "w3": tf.Variable(tf.random_normal([7*7*64,1024])),  ###Converting into Dense 
   "w4": tf.Variable(tf.random_normal([1024,num_classes]))} ###Final Class Probabilities

b={"b1":tf.Variable(tf.random_normal([32])),
   "b2":tf.Variable(tf.random_normal([64])),
   "b3":tf.Variable(tf.random_normal([1024])),
   "b4":tf.Variable(tf.random_normal([num_classes]))}

In [8]:
### Model Architechture


def Model_Conv(x,W,b,dropout):
    x=tf.reshape(x,shape=[-1,28,28,1])
    
    ##Convolving & Pooling
    c1=convolution_img(x,W["w1"],b["b1"],strides=1)
    c1=max_pooling_layer(c1,k=2)
    
    ##Convolving & Pooling
    
    c1=convolution_img(c1,W["w2"],b["b2"],strides=1)
    c1=max_pooling_layer(c1,k=2)
    
    
    ## Fully-Connected Layer
    c1=tf.reshape(c1,[-1,W["w3"].get_shape().as_list()[0]])
    c1=tf.matmul(c1,W["w3"])   ##     [-1,7*7*64]*[7*7*64,1024]
    c1=tf.add(c1,b["b3"])      ## [-1,1024]
    c1=tf.nn.relu(c1)
    
    ##Dropout
    c1=tf.nn.dropout(c1,dropout)
    
    ##Output
    c1=tf.matmul(c1,W["w4"])  ##  [-1,1024]*[1024,num_classes]
    c1=tf.add(c1,b["b4"])
    
    return c1                 ##  [-1,num_classes]

In [9]:
###Cost function Optimization & Model Evaluation

probs=Model_Conv(X,W,b,dropout=keep_prob)
predicted=tf.nn.softmax(probs)

optim=tf.train.AdamOptimizer(learning_rate=learning_rate)
cost_fn=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=probs,labels=Y))
training=optim.minimize(cost_fn)


actual=tf.argmax(Y,1)               ##   [[1,10,20],
                                          [5,7,30]]
preds=tf.argmax(predicted,1)

accuracy=tf.equal(actual,preds)
accuracy=tf.cast(accuracy,tf.float32)
accuracy=tf.reduce_mean(accuracy)



In [10]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

###Staring the Model training Session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = df.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(training, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost_fn, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) +   ",   Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: df.test.images[:256],
                                      Y: df.test.labels[:256],
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 85155.2578, Training Accuracy= 0.150
Step 20, Minibatch Loss= 10913.5088, Training Accuracy= 0.565
Step 40, Minibatch Loss= 4518.1465, Training Accuracy= 0.725
Step 60, Minibatch Loss= 5104.1011, Training Accuracy= 0.800
Step 80, Minibatch Loss= 3100.2927, Training Accuracy= 0.840
Step 100, Minibatch Loss= 3195.2039, Training Accuracy= 0.860
Step 120, Minibatch Loss= 2244.9724, Training Accuracy= 0.870
Step 140, Minibatch Loss= 897.0003, Training Accuracy= 0.915
Step 160, Minibatch Loss= 779.9336, Training Accuracy= 0.925
Step 180, Minibatch Loss= 1207.9463, Training Accuracy= 0.920
Step 200, Minibatch Loss= 1199.2750, Training Accuracy= 0.930
Step 220, Minibatch Loss= 902.1171, Training Accuracy= 0.930
Step 240, Minibatch Loss= 742.9398, Training Accuracy= 0.940
Step 260, Minibatch Loss= 1146.4990, Training Accuracy= 0.940
Step 280, Minibatch Loss= 1335.1587, Training Accuracy= 0.930
Step 300, Minibatch Loss= 1303.1082, Training Accuracy= 0.935
Step 320, Miniba